In [2]:
!pip install twikit


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
USERNAME = "VyuLeny"
EMAIL = "vyuleni4@gmail.com"
PASSWORD = 'vyulenimantap123'

In [25]:
import asyncio
from twikit import Client
import csv

client = Client(language='')
is_logged_in = False

async def twikitExec(filename, keywords, count, type): 
    global is_logged_in

    if not is_logged_in:
        await client.login(
            auth_info_1=USERNAME,
            auth_info_2=EMAIL,
            password=PASSWORD,
            cookies_file='cookies.json'
        )
        is_logged_in = True 

    query = " OR ".join(f'"{keyword}"' for keyword in keywords) + " lang:id"
    print(f'searching: {query}')

    csv_data = []
    fetched_ids = set()
    counted = 0
    isMoreThan20 = count > 20

    tweets = await client.search_tweet(query, type)
    for tweet in tweets:
        if tweet.id not in fetched_ids: 
            tweet_text = tweet.text.replace('\n', ' ')
            csv_data.append({
                'tweet_id': tweet.id,
                'username': tweet.user.screen_name,
                'text': tweet_text,
                'date': tweet.created_at
            })
            fetched_ids.add(tweet.id)
            counted += 1 

            if counted >= count:
                break

    while isMoreThan20 and (counted < count):
        try:
            if hasattr(tweets, 'next'):
                tweets = await tweets.next()
            else:
                break

            for tweet in tweets:
                if tweet.id not in fetched_ids: 
                    tweet_text = tweet.text.replace('\n', ' ')
                    csv_data.append({
                        'tweet_id': tweet.id,
                        'username': tweet.user.screen_name,
                        'text': tweet_text,
                        'date': tweet.created_at
                    })
                    fetched_ids.add(tweet.id)
                    counted += 1 

                    if counted >= count:
                        break
                    
            await asyncio.sleep(1)

        except Exception as e:
            if "429" in str(e):
                print("Rate limit exceeded. Waiting before retrying...")
                await asyncio.sleep(60)  # Wait for 60 seconds before retrying
            else:
                print(f"An error occurred: {e}")
                break

    csv_filename = f'{filename}.csv'

    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['tweet_id', 'username', 'text', 'date'], delimiter=';', quoting=csv.QUOTE_MINIMAL)
        writer.writeheader()
        writer.writerows(csv_data)

    print(f'Tweets have been saved to {csv_filename}')

In [27]:
from datetime import datetime

keywords = ["kopi enak", "kopi gaenak", "kopi terbaik", "pengalaman minum kopi", "rasa kopi", "kopi", "ulasan kopi", "review kopi", "rasa kopi", "kopi favorit"]
filename = f'tweets_{datetime.now().strftime("%Y%m%d%H%M%S")}'

await twikitExec(filename=filename, keywords=keywords, count=5500, type='Latest')


searching: "kopi enak" OR "kopi gaenak" OR "kopi terbaik" OR "pengalaman minum kopi" OR "rasa kopi" OR "kopi" OR "ulasan kopi" OR "review kopi" OR "rasa kopi" OR "kopi favorit" lang:id


Unauthorized: status: 401, message: "{"errors":[{"message":"Could not authenticate you","code":32}]}
"